<a href="https://colab.research.google.com/github/EUNJIKIM810/pytorch-tutorial/blob/main/pytprch_tutorial_%EC%8B%A0%EA%B2%BD%EB%A7%9D%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**신경망 모델 구성하기**

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

학습을 위한 장치 얻기

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


클래스 정의하기

신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화합니다. nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다.

In [4]:
class NeuralNetwork(nn.Module): #이 클래스는  nn.Module을 상속받아 정의
    def __init__(self):
        super().__init__() #부모 클래스인 nn.Module의 초기화 메서드를 호출
        self.flatten = nn.Flatten() #nn.Flatten(): 2D 이미지를 1D 벡터로 변환
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력합니다.

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device) #크기 (1, 28, 28)의 랜덤 데이터를 생성
logits = model(X) #입력 데이터 X를 신경망 모델에 통과시켜 **로짓(logits)**을 계산
pred_probab = nn.Softmax(dim=1)(logits) #로짓을 정규화하여 확률 분포로 변환
y_pred = pred_probab.argmax(1) #각 데이터 포인트에 대해 최대 확률을 가진 클래스의 인덱스를 반환
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


모델 계층(Layer)



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환

In [8]:
flatten = nn.Flatten() #차원이 1 이상인 모든 축을 하나로 병합하여 1D 벡터로 변환
flat_image = flatten(input_image) #input_image는 평탄화 대상이 되는 입력 텐서
print(flat_image.size()) #평탄화된 텐서의 크기를 출력

torch.Size([3, 784])


nn.Linear: 선형 계층 은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20) #nn.Linear는 선형 계층(Fully Connected Layer)을 생성
hidden1 = layer1(flat_image) #flat_image는 평탄화된 텐서. 선형 계층을 통과한 결과는 hidden1에 저장
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU: 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1) #ReLU 활성화 함수 적용
#음수 값 → 0.
#양수 값 → 그대로 유지.

print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0179, 0.2933, 0.0000, 0.2923, 0.0000, 0.2671, 0.0000, 0.1958, 0.2001,
         0.0000, 0.0000, 0.2123, 0.5755, 0.2038, 0.0000, 0.0000, 0.0000, 0.1443,
         0.0000, 0.0000],
        [0.2952, 0.3469, 0.3725, 0.0945, 0.0000, 0.2254, 0.0000, 0.0803, 0.0952,
         0.0000, 0.0000, 0.4485, 0.2969, 0.4050, 0.0000, 0.1533, 0.0000, 0.0829,
         0.0000, 0.0000],
        [0.0000, 0.0206, 0.0000, 0.1591, 0.0000, 0.5727, 0.0000, 0.3010, 0.0000,
         0.0000, 0.0000, 0.4600, 0.5027, 0.2293, 0.0000, 0.0035, 0.0000, 0.0938,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0179, 0.2933, 0.0000, 0.2923, 0.0000, 0.2671, 0.0000, 0.1958, 0.2001,
         0.0000, 0.0000, 0.2123, 0.5755, 0.2038, 0.0000, 0.0000, 0.0000, 0.1443,
         0.0000, 0.0000],
        [0.2952, 0.3469, 0.3725, 0.0945, 0.0000, 0.2254, 0.0000, 0.0803, 0.0952,
         0.0000, 0.0000, 0.4485, 0.2969, 0.4050, 0.0000, 0.1533, 0.0000, 0.0829,
         0.0000, 0.0000],
       

nn.Sequential 은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.

In [13]:
seq_modules = nn.Sequential(
    flatten, #이미지를 1D 벡터로 변환. -> 크기 변환: [batch_size, 1, 28, 28] → [batch_size, 28*28].
    layer1, #첫 번째 선형 계층.
    nn.ReLU(), #활성화 함수로 비선형성 추가.
    nn.Linear(20, 10) #두 번째 선형 계층으로 최종 출력 계산.
)
input_image = torch.rand(3,28,28) #크기 (3, 28, 28)의 랜덤 데이터를 생성
logits = seq_modules(input_image)

nn.Softmax

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

**모델 매개변수**

신경망 내부의 많은 계층들은 매개변수화(parameterize) 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0353,  0.0342,  0.0240,  ..., -0.0139, -0.0171, -0.0151],
        [-0.0019, -0.0263, -0.0137,  ..., -0.0265,  0.0273, -0.0255]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0333,  0.0009], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0313,  0.0327, -0.0441,  ..., -0.0078,  0.0359, -0.0370],
        [-0.0129,  0.0141,  0.0431,  ..., -0.0130,  0.0387,  0.0379]],
       device='cuda:0', grad_fn=<Sl